In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
import os
history_df= None

def generate_and_insert_plot( wb, summary_df, sheet_name, ticker='TICKER', strategies=["0.0"], intervals="1d", row=20, col=15):
    """
    Generuje wykres account_value z timestamp i wstawia go do arkusza w Workbooku.
    """
    # === 1. Generowanie wykresu ===
    curr_row=row
    for interval in intervals:
        plt.figure(figsize=(12, 6))
        for strategy in strategies:
            history_path = f'{simulation_data}/{ticker}- strategy {strategy}, interval {interval} - history .xlsx'
            history_df = pd.read_excel(history_path)
            strategy_description = summary_df[summary_df['Strategy'] == float(strategy)].iloc[0]['Description']
            if 'timestamp' in history_df.columns and 'account_value' in history_df.columns:
                if float(strategy) == float(0.0):
                    plt.plot(
                        history_df['timestamp'],
                        history_df['account_value'],
                        label=f"Strategy {strategy} - {strategy_description}",
                        color='black',
                        linewidth=1.5,
                        alpha=0.3)
                    
                else:
                    plt.plot(
                        history_df['timestamp'],
                        history_df['account_value'],
                        label=f"Strategy {strategy} - {strategy_description}",
                        linewidth=1.5
                    )
        image_path = os.path.join('Charts', f'{ticker}-{interval}.png')
        plt.title(f'Account Value Over Time\nTicker: {ticker} interwał: {interval} start date: {start_date}')
        plt.xlabel('Date')
        plt.ylabel('Account Value')
        plt.grid(True)
        plt.legend()

        ax = plt.gca()
        ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{int(x):,}'))
        plt.tight_layout()
        plt.savefig(image_path)
        plt.close()

        # === 2. Wstawienie do arkusza Excela ===
        if sheet_name not in wb.sheetnames:
            #print(f"❌ Arkusz '{sheet_name}' nie istnieje. - tworzę go")
            wb.create_sheet(title=sheet_name)

        ws = wb[sheet_name]
        img = ExcelImage(image_path)
        col_letter = get_column_letter(col)
        anchor_cell = f"{col_letter}{curr_row}"
        img.anchor = anchor_cell
        ws.add_image(img)
        curr_row+=30
        #print(f"✅ Wstawiono wykres dla {ticker} | Strategy {strategy} do arkusza '{sheet_name}' → {anchor_cell}")


simulation_data='output_simulations'

start_date = '2025-01-21'
start_date = '2024-01-01'
folder_output='output_simulations'
simulation_data=os.path.join(folder_output, start_date)




strategies=['0.0', '0.1', '0.3', '0.4']
intervals=["1d", "12h", "4h"]

sheet_output=f"Generated summaries for tickers- {start_date}.xlsx"
wb = load_workbook(sheet_output)

simulation_sheet_output=f'Simulations_summaries- {start_date}.xlsx'
summary_df = pd.read_excel(simulation_sheet_output)
tickers = summary_df['Ticker'].unique()
for ticker in tickers:
    generate_and_insert_plot(
        wb,
        summary_df,
        sheet_name=ticker,  # lub inna nazwa arkusza
        ticker=ticker,
        strategies= strategies,
        intervals=intervals,
        row=1,
        col=16
    )
wb.save(sheet_output)

In [ ]:
wb.save(sheet_output)